# Precipitation Rate Decadal Sampling Ablation
This notebook performs an ablation study on precipitation rate using CNN-MLP.

The notebook compares usage of the decadal and non-decadal sampler at different batch sizes

Please note that the parameters used are smaller than those that were submitted to github. This is for experimentation efficiency; the parameters are identical so the conclusions from these experiments are still valid

In [2]:
!git clone https://github.com/kabir-vats/Deep-Climate-Learning.git
!pip install -r Deep-Climate-Learning/requirements.txt

Cloning into 'Deep-Climate-Learning'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 89 (delta 38), reused 36 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 438.99 KiB | 10.71 MiB/s, done.
Resolving deltas: 100% (38/38), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━

In [3]:
import sys
sys.path.append('/kaggle/working/Deep-Climate-Learning')

In [4]:
from pytorch_lightning.loggers import WandbLogger
import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb-key")

wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kabirvats (kabirvats-uc-san-diego) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [16]:
from climate_prediction.data_loader import ClimateDataModule, ClimateDecadeDataModule
from climate_prediction.models.CNN_MLP import CNN_MLP
from climate_prediction.train import ClimateEmulationModule
from climate_prediction.util import compare_preds
import lightning.pytorch as pl
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

In [17]:
config = {
    "data": {
        "path": "/kaggle/input/cse151b-spring2025-competition/processed_data_cse151b_v2_corrupted_ssp245/processed_data_cse151b_v2_corrupted_ssp245.zarr",
        "input_vars": ["CO2", "SO2", "CH4", "BC", "rsdt"],
        "output_vars": ["pr"],
        "train_member_ids": [0,1,2],
        "val_member_id": 0,
        "target_member_id": 0,
        "train_ssps": ["ssp370", "ssp126"],
        "test_ssp": "ssp245",
        "test_months": 360,
        "val_months": 120,
        "batch_size": 64,
        "num_workers": 4,
        "sequence_length": 1,
    },
    "optimizer": {
        "lr": 2e-4,
        "scheduler": {
            "type": "OneCycle",
            "max_lr": 1e-3,
            "epochs": 30,
            "pct_start": 0.25,
            "anneal_strategy": 'cos',
            "div_factor": 10,
            "final_div_factor": 1e3,
            "cycle_momentum": False,
            'three_phase': True,
        }
    },
    "trainer": {
        "max_epochs": 30,
        "accelerator": "auto",
        "devices": "auto",
        "precision": "32",
        "deterministic": False,
        "num_sanity_val_steps": 0,
    },
    "model_type": "CNN_MLP",
    "model": {
        "lat": 48,
        "lon": 72,
        "channels_last": False,
        "seq_len": 1,
        "in_var_ids": ["a","b","c","d","e","f","g","h","9","10", "11", "12"],
        "out_var_ids": ["PR"],
        "hidden_size": 512,
        "seq_to_seq": False,
        "dropout": 0,
    },
    "loss": {
        "version": "Ramp",
        "wait_epochs": 15,
        "ramp_epochs": 2,
        "train_weights": {
            "tas": {"monthly_rmse": 0.1, "time_mean": 0.3, "time_std": 1.0},
            "pr": {"monthly_rmse": 0.1, "time_mean": 0.3, "time_std": 1},
        },
    },
    "seed": 42,
}

pl.seed_everything(config["seed"])

INFO: Seed set to 42


42

In [18]:
batch_sizes = [30, 60, 120]
for batch_size in batch_sizes:
    config["data"]["batch_size"] = batch_size
    checkpoint_callback = ModelCheckpoint(
        save_top_k=0
    )
    
    wandb_logger = WandbLogger(
        project="HK(N+1)",
        name=f'CNN_MLP_{batch_size}_Non_Decadal',
        config=config                    
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val/loss/crit2",   # the metric to monitor
        min_delta=0.0001,            # minimum change to qualify as improvement
        patience=10,                 # how many epochs to wait before stopping
        verbose=True,
        mode="min"                  # minimize the metric
    )
    
    datamodule = ClimateDataModule(**config["data"])
    model = CNN_MLP(**config["model"])
    lightning_module = ClimateEmulationModule(model, config["data"]["output_vars"], config["loss"], config["optimizer"])
    
    trainer = pl.Trainer(**config["trainer"], logger=wandb_logger, callbacks=[early_stop_callback, checkpoint_callback])
    trainer.fit(lightning_module, datamodule=datamodule)
    wandb.finish()

for batch_size in batch_sizes:
    config["data"]["batch_size"] = batch_size
    checkpoint_callback = ModelCheckpoint(
        save_top_k=0
    )
    
    wandb_logger = WandbLogger(
        project="HK(N+1)",
        name=f'CNN_MLP_{batch_size}_Decadal',
        config=config                    
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val/loss/crit2",   # the metric to monitor
        min_delta=0.0001,            # minimum change to qualify as improvement
        patience=10,                 # how many epochs to wait before stopping
        verbose=True,
        mode="min"                  # minimize the metric
    )
    
    datamodule = ClimateDecadeDataModule(**config["data"])
    model = CNN_MLP(**config["model"])
    lightning_module = ClimateEmulationModule(model, config["data"]["output_vars"], config["loss"], config["optimizer"])
    
    trainer = pl.Trainer(**config["trainer"], logger=wandb_logger, callbacks=[early_stop_callback, checkpoint_callback])
    trainer.fit(lightning_module, datamodule=datamodule)
    wandb.finish()

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


['pr']
['pr']


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type                    | Params | Mode 
---------------------------------------------------------------
0 | model      | CNN_MLP                 | 2.1 M  | train
1 | ramp1      | L1Loss                  | 0      | train
2 | ramp2      | AreaWeightedClimateLoss | 0      | train
3 | criterion1 | L1Loss                  | 0      | train
4 | criterion2 | AreaWeightedClimateLoss | 0      | train
---------------------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.366     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Using one cycle LR


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

[VAL]: Criterion1 (MSE)=1.0550, Criterion2 (Custom)=0.3262
[VAL] pr: RMSE=2.2271, Time-Mean RMSE=0.2764, Time-Stddev MAE=0.1977
[Epoch 30] Current LR (group 0): 1.0042009459537254e-07
[TRAIN]: Criterion1 (MSE)=0.9558, Criterion2 (Custom)=0.1194


INFO: `Trainer.fit` stopped: `max_epochs=30` reached.


[TRAIN] pr: RMSE=2.0162, Time-Mean RMSE=0.0083, Time-Stddev MAE=0.0369


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr,▂▃▄▅▆▇██▇▆▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▅▄▄▄▅▄▄▄▄▄▄▄▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss/crit1,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂
train/loss/crit2,█▆▄▃▃▃▃▃▃▃▃▃▃▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/pr/rmse,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂
train/pr/time_mean_rmse,█▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/pr/time_std_mae,██▅▅▄▄▄▄▄▄▄▄▄▄▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
val/loss,█▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss/crit1,█▄▂▃▁▃▂▂▂▁▁▁▁▁▁▁▂▃▃▃▃▃▃▃▃▃▃▃▃▃
